In [1]:
import pandas as pd
from sklearn import linear_model
import pickle
import lightgbm as lgb
import numpy as np
from scipy.stats import pearsonr as p
from tqdm import tqdm
import warnings 
warnings.filterwarnings('ignore')

In [13]:
train_df = pd.read_pickle('train_dropna.pkl')

In [2]:
with open('feature_corr_list.pkl', 'rb') as f:
    best_feats =pickle.load(f)[:10]

In [3]:
train_df = pd.read_pickle('train_dropna.pkl')
feat_top_10_time = pd.DataFrame()
for col in tqdm(best_feats):
    mapper = train_df.groupby(['time_id'])[col].mean().to_dict()
    feat_top_10_time[f'time_id_{col}_mean'] = train_df['time_id'].map(mapper)
    feat_top_10_time[f'time_id_{col}_mean'] = feat_top_10_time[f'time_id_{col}_mean'].astype(np.float16)

    mapper = train_df.groupby(['time_id'])[col].var().to_dict()
    feat_top_10_time[f'time_id_{col}_var'] = train_df['time_id'].map(mapper)
    feat_top_10_time[f'time_id_{col}_var'] = feat_top_10_time[f'time_id_{col}_var'].astype(np.float16)

feat_top_10_time.to_pickle("feat_top_10_time_corr.pkl")
# train_df = pd.read_pickle('train_dropna.pkl')

# for col in tqdm(best_feats):
#     mapper = train_df.groupby(['time_id'])[col].mean().to_dict()
#     train_df[f'time_id_{col}_mean'] = train_df['time_id'].map(mapper)
#     train_df[f'time_id_{col}_mean'] = train_df[f'time_id_{col}_mean'].astype(np.float16)

#     mapper = train_df.groupby(['time_id'])[col].var().to_dict()
#     train_df[f'time_id_{col}_var'] = train_df['time_id'].map(mapper)
#     train_df[f'time_id_{col}_var'] = train_df[f'time_id_{col}_var'].astype(np.float16)

#     mapper = train_df.groupby(['time_id'])[col].median().to_dict()
#     train_df[f'time_id_{col}_median'] = train_df['time_id'].map(mapper)
#     train_df[f'time_id_{col}_median'] = train_df[f'time_id_{col}_median'].astype(np.float16)


100%|██████████| 10/10 [00:02<00:00,  4.61it/s]


In [4]:
train_df

,time_id,stock_id,0,1,2,3,4,5,6,7,...,291,292,293,294,295,296,297,298,299,label
0,0,0,-0.209351,-1.010742,-0.287354,-0.600098,-0.579590,3.175781,0.916504,-0.233154,...,0.332520,1.214844,0.601074,-0.538086,1.053711,0.440674,-0.243530,1.708984,1.560547,-0.023621
1,0,1,-1.090820,-1.467773,-0.756348,-0.305908,-0.770508,0.167480,0.745605,-0.210815,...,-0.365234,-0.132935,-0.165527,-0.170044,-0.463867,0.429199,-0.531738,-0.297852,-0.554199,-0.016907
2,0,2,-0.171997,-0.359619,0.829102,0.323242,-0.086060,-0.853516,-0.770508,-0.521484,...,-0.168579,-0.566406,0.084717,-0.505859,-0.565918,-0.691895,-0.805664,0.023224,-0.358887,-0.004795
3,0,3,-0.089844,0.999512,0.130493,0.441650,0.366943,-2.294922,1.056641,-0.445557,...,-1.195312,0.698242,0.430908,-0.907227,0.580078,-0.145020,0.307373,-0.632812,0.038208,0.042908
4,0,4,0.231567,-0.168945,-0.376465,0.985840,-0.504883,-1.006836,0.308350,-0.220093,...,0.407959,-1.102539,0.532715,0.045685,-0.645020,0.381348,-0.258301,-0.053192,0.170044,0.017075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284582,727,2511,2.382812,0.966797,1.178711,1.184570,0.234619,-0.727539,1.342773,1.476562,...,0.457031,0.103149,0.418213,-2.035156,0.186279,0.628906,-0.391357,-0.178955,0.087952,-0.040802
1284583,727,2512,-0.343750,0.531250,2.455078,-1.387695,0.565918,0.487549,0.361816,-1.781250,...,-0.273193,1.776367,-0.925293,1.700195,1.781250,0.217773,0.169312,1.993164,1.558594,-0.030182
1284584,727,2513,-0.777344,-0.577148,1.499023,-1.237305,-1.456055,-1.187500,-0.364258,-1.907227,...,1.019531,1.125000,-0.534668,0.758789,1.097656,0.584473,1.157227,1.732422,1.332031,-0.147949
1284585,727,2534,2.312500,2.828125,0.207031,0.211670,1.645508,-1.622070,-1.346680,-0.047638,...,-0.736328,-2.097656,0.142578,1.021484,-1.815430,-0.230957,-0.287598,-1.707031,1.275391,0.003157


In [16]:
with open('feat_top10.pkl', 'rb') as f:
    best_feats =pickle.load(f)
train_df = pd.read_pickle('train_dropna.pkl')
feat_top_10_cross = pd.DataFrame()
for feat1 in best_feats:
    for feat2 in best_feats:
        if feat1 == feat2:
            continue
        else:
            feat_top_10_cross[f'{feat1}+{feat2}'] = train_df[feat1] + train_df[feat2]
            feat_top_10_cross[f'{feat1}-{feat2}'] = train_df[feat1] + train_df[feat2]
            feat_top_10_cross[f'{feat1}*{feat2}'] = train_df[feat1] + train_df[feat2]
feat_top_10_cross.to_pickle("feat_top_10_cross.pkl")

In [17]:
# # 定义一个函数，用于为每个分组添加之前所有时刻的"feature"列的平均值
with open('feat_top100.pkl', 'rb') as f:
    best_feats =pickle.load(f)
train_df = pd.read_pickle('train_dropna.pkl')
feat_top_100_stock_before = pd.DataFrame()

def add_mean_feature(group, col):
    # 计算之前所有时刻的"feature"列的平均值
    previous_features = group[col].expanding().mean().shift(1)
    group[f'mean_{col}'] = previous_features
    return group

def add_var_feature(group, col):
    # 计算之前所有时刻的"feature"列的平均值
    previous_features = group[col].expanding().var().shift(1)
    group[f'var_{col}'] = previous_features
    return group

# train_df.loc[(train_df['stock_id']==2512.0) & (train_df['time_id'] == 727.0)] 
train_df

,time_id,stock_id,0,1,2,3,4,5,6,7,...,291,292,293,294,295,296,297,298,299,label
0,0,0,-0.209351,-1.010742,-0.287354,-0.600098,-0.579590,3.175781,0.916504,-0.233154,...,0.332520,1.214844,0.601074,-0.538086,1.053711,0.440674,-0.243530,1.708984,1.560547,-0.023621
1,0,1,-1.090820,-1.467773,-0.756348,-0.305908,-0.770508,0.167480,0.745605,-0.210815,...,-0.365234,-0.132935,-0.165527,-0.170044,-0.463867,0.429199,-0.531738,-0.297852,-0.554199,-0.016907
2,0,2,-0.171997,-0.359619,0.829102,0.323242,-0.086060,-0.853516,-0.770508,-0.521484,...,-0.168579,-0.566406,0.084717,-0.505859,-0.565918,-0.691895,-0.805664,0.023224,-0.358887,-0.004795
3,0,3,-0.089844,0.999512,0.130493,0.441650,0.366943,-2.294922,1.056641,-0.445557,...,-1.195312,0.698242,0.430908,-0.907227,0.580078,-0.145020,0.307373,-0.632812,0.038208,0.042908
4,0,4,0.231567,-0.168945,-0.376465,0.985840,-0.504883,-1.006836,0.308350,-0.220093,...,0.407959,-1.102539,0.532715,0.045685,-0.645020,0.381348,-0.258301,-0.053192,0.170044,0.017075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284582,727,2511,2.382812,0.966797,1.178711,1.184570,0.234619,-0.727539,1.342773,1.476562,...,0.457031,0.103149,0.418213,-2.035156,0.186279,0.628906,-0.391357,-0.178955,0.087952,-0.040802
1284583,727,2512,-0.343750,0.531250,2.455078,-1.387695,0.565918,0.487549,0.361816,-1.781250,...,-0.273193,1.776367,-0.925293,1.700195,1.781250,0.217773,0.169312,1.993164,1.558594,-0.030182
1284584,727,2513,-0.777344,-0.577148,1.499023,-1.237305,-1.456055,-1.187500,-0.364258,-1.907227,...,1.019531,1.125000,-0.534668,0.758789,1.097656,0.584473,1.157227,1.732422,1.332031,-0.147949
1284585,727,2534,2.312500,2.828125,0.207031,0.211670,1.645508,-1.622070,-1.346680,-0.047638,...,-0.736328,-2.097656,0.142578,1.021484,-1.815430,-0.230957,-0.287598,-1.707031,1.275391,0.003157


In [18]:
for col in tqdm(best_feats):
    train_df = train_df.groupby("stock_id").apply(add_mean_feature, col=col)
    train_df = train_df.groupby("stock_id").apply(add_var_feature, col=col)
    train_df[f'mean_{col}'] = train_df[f'mean_{col}'].fillna(0)
    train_df[f'var_{col}'] = train_df[f'var_{col}'].fillna(0)

100%|██████████| 100/100 [47:15<00:00, 28.35s/it]


In [20]:
for col in tqdm(best_feats):
    feat_top_100_stock_before[f'mean_{col}'] = train_df[f'mean_{col}']
    feat_top_100_stock_before[f'var_{col}'] = train_df[f'var_{col}']

100%|██████████| 100/100 [00:01<00:00, 74.82it/s]


In [39]:
feat_top_100_stock_before.to_pickle("feat_top_100_stock_before.pkl")

In [44]:
train_df = pd.read_pickle('train_dropna.pkl')
with open('feat_top100.pkl', 'rb') as f:
    best_feats =pickle.load(f)
lagging_df1 = pd.DataFrame()
for fe in tqdm(best_feats):
    lagging_df1[f"lagging_1_{fe}_rate"] = train_df.groupby("stock_id")[fe].diff(1)
    lagging_df1[f"lagging_1_{fe}_rate"] = lagging_df1[f"lagging_1_{fe}_rate"].fillna(0).astype(np.float16)
lagging_df1.to_pickle("lagging_df1.pkl")
lagging_df2 = pd.DataFrame()
for fe in tqdm(best_feats):
    lagging_df2[f"lagging_2_{fe}_rate"] = train_df.groupby("stock_id")[fe].diff(2)
    lagging_df2[f"lagging_2_{fe}_rate"] = lagging_df2[f"lagging_2_{fe}_rate"].fillna(0).astype(np.float16)
lagging_df2.to_pickle("lagging_df2.pkl")
lagging_df3 = pd.DataFrame()
for fe in tqdm(best_feats):
    lagging_df3[f"lagging_3_{fe}_rate"] = train_df.groupby("stock_id")[fe].diff(3)
    lagging_df3[f"lagging_3_{fe}_rate"] = lagging_df3[f"lagging_3_{fe}_rate"].fillna(0).astype(np.float16)
lagging_df3.to_pickle("lagging_df3.pkl")


100%|██████████| 100/100 [02:05<00:00,  1.25s/it]
